<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/analysis/mean_reversion_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install finance-datareader

In [22]:
import pandas as pd
import plotly.graph_objects as go
import FinanceDataReader as fdr

# Download Nasdaq data
nasdaq = fdr.DataReader('IXIC', '1980-01-01')['Close']
nasdaq.index = pd.to_datetime(nasdaq.index, utc=True)
log_nasdaq = np.log(nasdaq)

# Define window sizes
windows = [20, 60, 120, 250]
windows = [500]

# Calculate mean reversion rates
mean_reversion_rates = {}
for window in windows:
    mean_reversion_rates[window] = []
    for i in range(window, len(nasdaq)):
        window_mean = nasdaq.iloc[i - window : i].mean()
        current_price = nasdaq.iloc[i]
        reversion_rate = (current_price - window_mean) / window_mean
        mean_reversion_rates[window].append(reversion_rate)

# Create figure
fig = go.Figure()

# Add traces for mean reversion rates
for window, rates in mean_reversion_rates.items():
    fig.add_trace(go.Scatter(x=nasdaq.index[window:], y=rates, mode='lines', name=f"{window}-day window"))

# Add trace for close price on the right y-axis
fig.add_trace(go.Scatter(x=nasdaq.index, y=log_nasdaq, mode='lines', name='Close Price', yaxis='y2', line=dict(color='red')))

# Update layout
fig.update_layout(
    title="Mean Reversion Rates for Nasdaq Index",
    xaxis_title="Date",
    yaxis_title="Mean Reversion Rate",
    legend_title="Window Size",
    xaxis_rangeslider_visible=True,
    yaxis2=dict(
        title="Close Price",
        overlaying="y",
        side="right",
        position=1,
        showgrid=False,
        showticklabels=True,
        linecolor='red',
        tickfont=dict(color='red'),
        tickmode="auto",
        zeroline=False
    )
)

# Show plot
fig.show()
